In [1]:
"""
환경: 파이썬 3.6
설치: pip install -r requirements.txt
"""

import pandas as pd

data_df = pd.read_csv('shopping-cacls.data.tsv', sep='\t',
                      names=['seq', 'category', 'brand', 'manufacturer', 'price', 'product_url', 'name'])
data_df = data_df.fillna('')
print(len(data_df))

61113


In [2]:
import numpy as np

In [3]:
# 데이터 로드 및 학습, 테스트 데이터 분리
msk = np.random.rand(len(data_df)) < 0.9
train_df = data_df[msk]
test_df = data_df[~msk]

print(f'train data length: {len(train_df)}')
print(f'test data length: {len(test_df)}')

train data length: 54964
test data length: 6149


In [101]:
# 학습데이터로 상풍명, 브랜드, 제조사, 카테고리 데이터 사용
# f1: 상품명 -> 카테고리
# f2: 브랜드 -> 카테고리
# f3: 제조사 -> 카테고리
# 3가지 머신을 naive bayes로 학습하며 예측은 세 머신에서 결과로 나오는 각 확률 값을 모두 곱하여 가장 큰 확률 값으로 분류함
train_name_data = train_df['name'].tolist()
train_brand_data = train_df['brand'].tolist()
train_manu_data = train_df['manufacturer'].tolist()
train_target_data = train_df['category'].tolist()
print(train_name_data[:10])
print(train_brand_data[:10])
print(train_name_data[:10])
print(train_target_data[:10])

['[해외배송]B002KQ5T9Y/Magnum Mens Elite Spider 8.0', 'Palladium  Palladium Womens Pampa Hi Canvas Boot', '[fashionplus][미소페]남성 워커 털부츠 021436009 굽1.5cm 흑색', '[패션플러스][소다] 소다 남성 데져트 캐주얼부츠 ASC315 KA20밤색/NA33진베이지 (3CM)', '[정품]Justin Boots Men’s 3001 Farm & Ranch 10 Boot Roper Toe Rubber Outsole', 'Smith Ski und Snowboardhelm Premise Bluetooth schwarz glnzend', 'Di Coletti over  di colletti side Gore boots', 'Church  CHURCH   S KETSBY KETSBY BK (BLACK / UK 7', '[PALLADIUM] Pallabrouse BGY EXTX,73690-967/FLPL6S1U26/팔라디움 팔라부르즈', '필리오 버클코디 앵글부츠 정장구두(브라운) 옥스퍼드 남성화 신사화 키높이구두 드레스슈즈']
['매그넘', '팔라디움', '미소페', '소다', '', '스미스옵틱스', '', '', '팔라디움', '']
['[해외배송]B002KQ5T9Y/Magnum Mens Elite Spider 8.0', 'Palladium  Palladium Womens Pampa Hi Canvas Boot', '[fashionplus][미소페]남성 워커 털부츠 021436009 굽1.5cm 흑색', '[패션플러스][소다] 소다 남성 데져트 캐주얼부츠 ASC315 KA20밤색/NA33진베이지 (3CM)', '[정품]Justin Boots Men’s 3001 Farm & Ranch 10 Boot Roper Toe Rubber Outsole', 'Smith Ski und Snowboardhelm Premise Bluetooth schwarz glnzend', 'Di Co

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_name_data)
print(X_train_counts.shape)
print(X_train_counts[0])

(54964, 49385)
  (0, 25020)	1
  (0, 14668)	1
  (0, 19995)	1
  (0, 19559)	1
  (0, 10604)	1
  (0, 48775)	1


In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

# tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
# X_train_tf = tf_transformer.transform(X_train_counts)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

print(X_train_counts[0])
print('-' * 35)
print(X_train_tfidf[0])

  (0, 25020)	1
  (0, 14668)	1
  (0, 19995)	1
  (0, 19559)	1
  (0, 10604)	1
  (0, 48775)	1
-----------------------------------
  (0, 48775)	0.343259304029
  (0, 10604)	0.52330385116
  (0, 19559)	0.415923143023
  (0, 19995)	0.191318463928
  (0, 14668)	0.453161428867
  (0, 25020)	0.439745418856


In [26]:
# 학습 !
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
name_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 10))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', MultinomialNB(alpha=1e-2))])

brand_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 10))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', MultinomialNB(alpha=1e-2))])

manu_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 10))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', MultinomialNB(alpha=1e-2))])

name_clf = name_clf.fit(train_name_data, train_target_data)
brand_clf = brand_clf.fit(train_brand_data, train_target_data)
manu_clf = manu_clf.fit(train_manu_data, train_target_data)

In [27]:
test_data = test_df['name'].tolist()
test_target_data = test_df['category'].tolist()
print(test_data[:10])
print(test_target_data[:10])

['Church  CHURCH   S BLETSOE R BLETSOE R BK (BLACK', 'Thorogood Mens 6 Safety Moc Toe Tobacco Boot', '[[금강]랜드로바][금강]랜드로바 버팔로 남성 플렉시블 보트 처커MBB0809GR17/FO57 (영등포점B)', '@MustHaveitem@ [리비티  KA3908]에어캡 NonSlip W 45도 남녀공용 경량 워킹슈즈 트레킹화 방한화 부츠 털단화 스니커즈', '369몰 U7 남성 운동화 단화 슬립온 구두 가을 로퍼 멋쟁이', '[신세계백화점]탠디  (신세계센텀점)탠디 남성 하이탑 51762 G-176 진밤고트버니쉬풀업 3CM', '후쿠야마 고무 패션 레인 프린트 부츠 마일로즈20 블랙 L (2015-04-15)', '지벡 XEBEC 장화 85706 10 네이비 4L(관/부가세 미포함)#TJP387', '[닥터마틴 18% 제휴할인] 플로라 첼시 부츠 (FLORA CHELSEA BOOT - BLACK) [DM14649001] DM', 'Clarks Desert Boot#TT387']
['부츠', '부츠', '부츠', '부츠', '부츠', '부츠', '부츠', '부츠', '부츠', '부츠']


In [45]:
predicted_by_name = name_clf.predict(test_data)
print(np.mean(predicted_by_name == test_target_data))

predicted_by_brand = brand_clf.predict(test_data)
print(np.mean(predicted_by_brand == test_target_data))

predicted_by_manu = manu_clf.predict(test_data)
print(np.mean(predicted_by_manu == test_target_data))

0.852658968938
0.327207676045
0.280045535859


In [92]:
# 예측!
print(name_clf.classes_)
print(brand_clf.classes_)
print(manu_clf.classes_)
results_p = np.multiply(np.multiply(name_clf.predict_proba(test_data), brand_clf.predict_proba(test_data)), manu_clf.predict_proba(test_data))

['기능화' '모카신/털신' '보트슈즈' '부츠' '샌들' '스니커즈' '슬리퍼' '슬립온' '실내화' '운동화' '워커' '웰트화'
 '정장구두']
['기능화' '모카신/털신' '보트슈즈' '부츠' '샌들' '스니커즈' '슬리퍼' '슬립온' '실내화' '운동화' '워커' '웰트화'
 '정장구두']
['기능화' '모카신/털신' '보트슈즈' '부츠' '샌들' '스니커즈' '슬리퍼' '슬립온' '실내화' '운동화' '워커' '웰트화'
 '정장구두']


In [93]:
import operator
results_cls = []
for result_p in results_p:
    index, value = max(enumerate(result_p), key=operator.itemgetter(1))
    result_cls = name_clf.classes_[index]
    results_cls.append(result_cls)

In [94]:
# 평가!
print(len(results_cls))
print(len(test_target_data))
print(np.array(results_cls) == test_target_data)
print(np.mean(np.array(results_cls) == test_target_data))

6149
6149
[ True  True False ...,  True  True  True]
0.830704179541


In [103]:
from sklearn import metrics
print(metrics.classification_report(test_target_data, predicted, target_names=list(set(test_target_data))))
print(metrics.confusion_matrix(test_target_data, predicted))

             precision    recall  f1-score   support

        웰트화       0.97      0.95      0.96       496
       보트슈즈       0.81      0.90      0.85       360
     모카신/털신       0.78      0.80      0.79       490
         샌들       0.87      0.82      0.85       559
        기능화       0.87      0.87      0.87       469
         부츠       0.87      0.82      0.85       503
         워커       0.89      0.86      0.88       496
       스니커즈       0.83      0.80      0.82       510
        운동화       0.93      0.93      0.93       414
       정장구두       0.85      0.86      0.86       513
        실내화       0.79      0.83      0.81       461
        슬리퍼       0.85      0.85      0.85       375
        슬립온       0.78      0.83      0.80       503

avg / total       0.85      0.85      0.85      6149

[[469   0   3   2   1   1   4   1   2   1   3   2   7]
 [  0 323  13   6   0   0   1   5   3   1   2   2   4]
 [  0  31 391   7   0   2   3  16   0   0   9   3  28]
 [  6   7  13 460   4   1   0   3   1

In [60]:
# SVM으로 학습
from sklearn.linear_model import SGDClassifier
svm_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 5))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=6, random_state=42))])
svm_clf = svm_clf.fit(train_data, train_target_data)

svm_predicted = svm_clf.predict(test_data)
print(np.mean(svm_predicted == test_target_data))

0.817611075338


In [95]:
# 새로운 테스트 데이터
test_df2 = pd.read_csv('shopping-ca-cls.data.test.tsv', sep='\t', names=['seq', 'category', 'brand', 'manufacturer', 'price', 'product_url', 'name'])
print(f'train data length: {len(test_df2)}')

train data length: 1113


In [96]:
test_data2 = test_df2['name'].tolist()
test_target_data2 = test_df2['category'].tolist()
print(test_data2[:10])
print(test_target_data2[:10])

['SHOES > 유러피언 버클롱부츠', '[레저포유] 마틴화/웰트화 No2051-국산수제 블랙 테러화 (반목 긴목) 4종! 천연소가죽 검정테러화 정글화 기동화 테러화 작전화 구조화 전투화 군화 경비화 부츠 워커 전술화', '[해외]팀버랜드 남성 척카 부츠 23061', '[3일배송] 울버린 1000마일 W00137 Wolverine 1000 Mile Boot CORDOVAN NO.8', '[ 추가 1~3%할인] 쏘렐 남성 부츠 카리부/Sorel Caribou II Boot', '◈JCORPS/제이콥스◈ 겨울컬렉션 버렌퍼미드탑  남성로퍼/스니커즈/워커/부츠/남성구두 온라인판매중!', '브랜드없음 스파이더 택티컬 전술부츠 [Asiaon] Spider Tactical Boots - 아시아온 스파이더 택티컬 전술', 'Funtasma by Pleaser Mens Bloody-06/W Slip-On', '톰브라운 부츠[st 레플리카] 웡팁워커', '[BOYLONDON]보이런던 가을겨울신상 토마미드탑 기모 스웨이드 스니커즈 캐주얼 단화 구두 온라인판매중']
['부츠', '부츠', '부츠', '부츠', '부츠', '부츠', '부츠', '부츠', '부츠', '부츠']


In [104]:
# 평가!!
results_p2 = np.multiply(np.multiply(name_clf.predict_proba(test_data2), brand_clf.predict_proba(test_data2)), manu_clf.predict_proba(test_data2))
import operator
results_cls2 = []
for result_p in results_p2:
    index, value = max(enumerate(result_p), key=operator.itemgetter(1))
    results_cls2.append(name_clf.classes_[index])
print(np.mean(np.array(results_cls2) == test_target_data2))

0.595687331536


In [105]:
# 평가!!
predicted2 = name_clf.predict(test_data2)
print(np.mean(predicted2 == test_target_data2))

0.617250673854


In [106]:
# 최적의 파라미터 찾기
from sklearn.grid_search import GridSearchCV

parameters = {'vect__ngram_range': [(1,i) for i in range(1, 10)],
              'tfidf__use_idf': (False, True),
              'clf__alpha': (1e-2, 1e-3, 1e-4)}

/Users/LCH/.pyenv/versions/shopping_category_classification/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/LCH/.pyenv/versions/shopping_category_classification/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [91]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(train_data, train_target_data)

In [92]:
best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

print(score)

clf__alpha: 0.01
tfidf__use_idf: False
vect__ngram_range: (1, 9)
0.8332999398918053
